# Model Replication

In [1]:
# imports
import pandas as pd
import numpy as np
import pprint
import statsmodels.api as sm
from statsmodels.stats import multitest
from scipy import stats
from scipy.stats import levene, norm
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib
import seaborn as sns
from matplotlib.colors import to_rgb
import colorsys
import itertools

plt.rcParams['figure.dpi'] = 600
colors = plt.colormaps["Dark2"].colors
from tqdm import tqdm

# silence warnings because the NN has a lot of them
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from optuna import create_study
from optuna.trial import Trial
from tensorflow.keras import backend as K
import random

# for saving model params
import pickle

# for feature importance
import shap
from sklearn.metrics import mean_squared_error
import copy
import ast

In [2]:
### CONDITION-LEVEL DATA
df_condition_level_data = pd.read_csv('../outputs/condition_level_group_advantage_with_ivs.csv') # mark's version of the cleaning

# add in categorical task types
df_condition_data_with_mcgrath_types = pd.read_csv('../outputs/condition_level_group_advantage_with_ivs_and_categories.csv')
mcgrath_cols = df_condition_data_with_mcgrath_types.filter(like="Type").columns
df_condition_data_with_mcgrath_types = df_condition_data_with_mcgrath_types[["task"] + list(mcgrath_cols)].drop_duplicates()
# rename the mcgrath columns to have the suffix "_categorical"
df_condition_data_with_mcgrath_types = df_condition_data_with_mcgrath_types.rename(columns={col: col + "_categorical" for col in mcgrath_cols})
df_condition_level_data = df_condition_level_data.merge(df_condition_data_with_mcgrath_types, on="task", how="left")

In [3]:
# inspect data, just to get a sense of the range of values.
df_condition_level_data[["task", "strong", "High", "Medium", "Low"]].sort_values("strong", ascending=False)

,task,strong,High,Medium,Low
39,Putting Food Into Categories,1.659080,0,1,0
113,Word Construction,1.627765,1,0,0
80,Unscramble Words,1.627760,0,1,0
37,Putting Food Into Categories,1.606875,0,0,1
41,Putting Food Into Categories,1.534374,1,0,0
...,...,...,...,...,...
5,Advertisement Writing,0.443447,1,0,0
3,Advertisement Writing,0.417235,0,1,0
89,Whac a Mole,0.331275,1,0,0
45,Random Dot Motion,0.297844,0,1,0


In [4]:
df_observation_level_data = pd.read_csv('../outputs/observation_level_dv_with_composition.csv')

In [5]:
FOCAL_DEMOGRAPHIC_COLS = ['IRCS_GS', 'IRCS_GV', 'IRCS_IB', 'IRCS_IR', 'IRCS_IV', 'IRCS_RS',
						'RME', 'CRT', 'birth_year', 'education_level', 'income_max', 'political_fiscal', 'political_party', 'political_social',
						'gender', 'race', 'marital_status']

COMPOSITION_IVS = [col for col in df_observation_level_data.columns if any(col.startswith(prefix) for prefix in FOCAL_DEMOGRAPHIC_COLS)]
COMPOSITION_IVS = COMPOSITION_IVS + ["playerCount", "Low", "Medium", "High"]

# p-value check: Within-Task Type Variance

"...there is almost as much within-category variation as exists overall."

In [6]:
mcgrath_cat_cols = df_condition_data_with_mcgrath_types.filter(like="_cat").columns
cat_col = df_condition_level_data[mcgrath_cat_cols].idxmax(axis=1)
df_condition_level_data_with_cat = df_condition_level_data.assign(cat_col=cat_col)

# Aggregate and summarize data by category
categorical_condition_summary = df_condition_level_data_with_cat.groupby("cat_col").agg({
	"strong": ["mean", "std", "count"],
	"weak": ["mean", "std", "count"]
}).reset_index()

# categorical level summary
display(categorical_condition_summary)

# display the strong and weak values for the entire dataset
display(df_condition_level_data["strong"].describe())
display(df_condition_level_data["weak"].describe())

strong_vals = df_condition_level_data["strong"]
weak_vals = df_condition_level_data["weak"]

strong_test_stats = []
weak_test_stats = []
strong_p_vals = []
weak_p_vals = []
df1_list = []
df2_list = []

# Iterate over each unique category to apply Levene's test
for cat in categorical_condition_summary['cat_col']:
	category_data = df_condition_level_data_with_cat[df_condition_level_data_with_cat['cat_col'] == cat]
	
	if category_data["strong"].count() > 6:
		cat_strong_vals = category_data["strong"]
		cat_weak_vals = category_data["weak"]

		# Levene's test comparing group variance to overall variance
		"""
		The Levene test tests the null hypothesis that all input samples 
		are from populations with equal variances. Levene’s test is an 
		alternative to Bartlett’s test bartlett in the case where there 
		are significant deviations from normality.

		Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html

		"""
		W_stat_strong, p_val_strong = levene(cat_strong_vals, strong_vals)
		W_stat_weak, p_val_weak = levene(cat_weak_vals, weak_vals)

		# degrees of freedom
		"""
		The degree of freedom df1 is obtained by calculating the number of groups minus 1,
		the degree of freedom df2 is obtained by calculating the number of cases minus the 
		number of groups.  Source: https://datatab.net/tutorial/levene-test
		"""
		NUM_GROUPS = 2
		df1_list.append(NUM_GROUPS - 1)
		df2_list.append(len(cat_strong_vals) + len(strong_vals) - NUM_GROUPS)
		assert(len(cat_weak_vals) == len(cat_strong_vals)) # means we only need to calculate df once
		assert(len(strong_vals) == len(weak_vals))

		strong_test_stats.append(W_stat_strong)
		weak_test_stats.append(W_stat_weak)
		strong_p_vals.append(p_val_strong)
		weak_p_vals.append(p_val_weak)
	else:
		strong_test_stats.append(None)
		weak_test_stats.append(None)
		strong_p_vals.append(None)
		weak_p_vals.append(None)
		df1_list.append(None)
		df2_list.append(None)

result_summary = categorical_condition_summary.assign(
	strong_W_stat=strong_test_stats, 
	strong_p_value=strong_p_vals, 
	weak_W_stat=weak_test_stats, 
	weak_p_value=weak_p_vals,
	df1 = df1_list,
	df2 = df2_list
)
display(result_summary)

cat_col    strong                  \
                                                mean       std count   
0            Type 2 (Generate)_categorical  1.040283  0.333841    30   
1        Type 3 (Intellective)_categorical  0.953430  0.227294    66   
2     Type 4 (Decision-Making)_categorical  0.689811  0.060508     6   
3  Type 5 (Cognitive Conflict)_categorical  0.671171  0.104386     6   
4         Type 8 (Performance)_categorical  0.837984  0.366415    12   

       weak                  
       mean       std count  
0  1.725222  0.808108    30  
1  1.547579  0.586886    66  
2  1.024908  0.122569     6  
3  1.207868  0.117462     6  
4  1.521116  0.608324    12

count    120.000000
mean       0.936305
std        0.280842
min        0.261190
25%        0.801759
50%        0.920559
75%        1.001102
max        1.659080
Name: strong, dtype: float64

count    120.000000
mean       1.546224
std        0.640559
min        0.587855
25%        1.116857
50%        1.304525
75%        1.720179
max        3.730325
Name: weak, dtype: float64

cat_col    strong                  \
                                                mean       std count   
0            Type 2 (Generate)_categorical  1.040283  0.333841    30   
1        Type 3 (Intellective)_categorical  0.953430  0.227294    66   
2     Type 4 (Decision-Making)_categorical  0.689811  0.060508     6   
3  Type 5 (Cognitive Conflict)_categorical  0.671171  0.104386     6   
4         Type 8 (Performance)_categorical  0.837984  0.366415    12   

       weak                 strong_W_stat strong_p_value weak_W_stat  \
       mean       std count                                            
0  1.725222  0.808108    30      1.011807       0.316112    2.139709   
1  1.547579  0.586886    66      3.383290       0.067472    0.426800   
2  1.024908  0.122569     6           NaN            NaN         NaN   
3  1.207868  0.117462     6           NaN            NaN         NaN   
4  1.521116  0.608324    12      3.606798       0.059760    0.074308   

  weak_p_value  df1    df2  
                            
0     0.145649  1.0  148.0  
1     0.514379  1.0  184.0  
2          NaN  NaN    NaN  
3          NaN  NaN    NaN  
4     0.785598  1.0  130.0

## Complexity interaction patterns

For instance, in Whac-A-Mole, groups display weak advantage at low levels of complexity, but their advantage decreases as the complexity increases. Likewise, the Putting Food into Categories task shows a strong interaction with group size, in which larger groups demonstrate more advantage than smaller ones, regardless of the level of complexity. But all of these interaction patterns are heterogeneous across the 20 tasks in our corpus.

In [7]:
for task in df_observation_level_data["task"].unique():

	print("****Results for Task: " +  task + " ****")

	task_data = df_observation_level_data[df_observation_level_data["task"] == task]
	
	# turn the categories ["Low", "Medium", and "High"] into an ordinal variable in the df task_data
	task_data['task_complexity'] = 1 * task_data['Low'] + 2 * task_data['Medium'] + 3 * task_data['High']

	# Fit a regression of Group Advantage ~ Task Complexity + Group Size with a random effect for group ID
	strong_model = sm.MixedLM.from_formula("strong ~ task_complexity + playerCount", groups="playerIds", data=task_data)
	strong_result = strong_model.fit()
	print("===Strong Group Advantage===")
	print(strong_result.summary())

	weak_model = sm.MixedLM.from_formula("weak ~ task_complexity + playerCount", groups="playerIds", data=task_data)
	weak_result = weak_model.fit()
	print("===Weak Group Advantage===")
	print(weak_result.summary())

****Results for Task: Unscramble Words ****
===Strong Group Advantage===
          Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  strong  
No. Observations:   159      Method:              REML    
No. Groups:         53       Scale:               0.1029  
Min. group size:    3        Log-Likelihood:      -51.8359
Max. group size:    3        Converged:           No      
Mean group size:    3.0                                   
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept        1.202    0.101 11.920 0.000  1.004  1.400
task_complexity  0.083    0.031  2.674 0.008  0.022  0.144
playerCount     -0.023    0.017 -1.363 0.173 -0.057  0.010
playerIds Var    0.000    0.039                           

===Weak Group Advantage===
         Mixed Linear Model Regression Results
Model:             MixedLM Dependent 

# Run Models

### Data Structure for Storing the Final Models & Outputs

In [8]:
base_model = {
			"OLS": {
				"Wave 1": {"strong": None, "weak": None},
				"Wave 2": {"strong": None, "weak": None}
			},
			"E-Net": {
				"Wave 1": {"strong": None, "weak": None},
				"Wave 2": {"strong": None, "weak": None}
			},
			"NN": {
				"Wave 1": {"strong": None, "weak": None},
				"Wave 2": {"strong": None, "weak": None}
			}
		}

In [9]:
try:
	# read the models and results from the previous save
	with open('../outputs/models.pkl', 'rb') as f:
		MODELS = pickle.load(f)
	with open('../outputs/model_r2_results.pkl', 'rb') as f:
		MODEL_R2_RESULTS = pickle.load(f)
	with open('../outputs/model_rmse_results.pkl', 'rb') as f:
		MODEL_RMSE_RESULTS = pickle.load(f)

except FileNotFoundError:
	# if the file is not found, we will create the MODELS and MODEL_R2_RESULTS dictionaries
	MODELS = {
		"Task Space": copy.deepcopy(base_model),
		"McGrath Categorical": copy.deepcopy(base_model),
		"McGrath Subspace": copy.deepcopy(base_model),
		"Steiner Subspace": copy.deepcopy(base_model),
		"Laughlin Subspace": copy.deepcopy(base_model)
	}

	# create a dictionary with the sake keys as MODELS to story R2 results
	MODEL_R2_RESULTS = copy.deepcopy(MODELS)

	# create a dictionary with the sake keys as MODELS to story RMSE results
	MODEL_RMSE_RESULTS = copy.deepcopy(MODELS)

In [10]:
# save MODELS and MODEL_R2_RESULTS
def save_current_model_and_results_version(MODELS, MODEL_R2_RESULTS, MODEL_RMSE_RESULTS):
	with open('../outputs/models.pkl', 'wb') as f:
		pickle.dump(MODELS, f)
	with open('../outputs/model_r2_results.pkl', 'wb') as f:
		pickle.dump(MODEL_R2_RESULTS, f)
	with open('../outputs/model_rmse_results.pkl', 'wb') as f:
		pickle.dump(MODEL_RMSE_RESULTS, f)

In [11]:
save_current_model_and_results_version(MODELS, MODEL_R2_RESULTS, MODEL_RMSE_RESULTS)

# Linear Models

In [12]:
# Hold out 10 tasks
wave1_df_train = df_condition_level_data[df_condition_level_data["wave"] == 1] ## train on this
wave1_df_test = df_condition_level_data[df_condition_level_data["wave"] > 1] ## test on everything else

# Hold out 5 tasks
wave12_df_train = df_condition_level_data[df_condition_level_data["wave"] <= 2] ## then train on this
wave12_df_test = df_condition_level_data[df_condition_level_data["wave"] == 3] ## test on everything else

In [13]:
# datastructure for the training and testing data
TRAIN_TEST_DATA = {
	"Wave 1": {
		"train": wave1_df_train,
		"test": wave1_df_test
	},
	"Wave 2": {
		"train": wave12_df_train,	
		"test": wave12_df_test
	}
}

# save TRAIN_TEST_DATA as a pickle file
with open('../outputs/train_test_data.pkl', 'wb') as f:
	pickle.dump(TRAIN_TEST_DATA, f)

In [14]:
# lists of IV's
CONTROLS = ["playerCount", "Low", "Medium", "High"]

basic_IVs = [col for col in df_condition_level_data.columns if 
	"categorical" not in col and "task" not in col and "strong" not in col and "weak" not in col and "wave" not in col]

mcgrath_continuous = [col for col in basic_IVs if "Type" in col] + ["Conceptual-Behavioral"] + CONTROLS

steiner_continuous = ["Divisible-Unitary", "Maximizing", "Optimizing"] + CONTROLS

laughlin_continuous = ["Decision Verifiability", "Shared Knowledge", "Within-System Solution", "Answer Recognizability", "Time Solvability", "Intellective-Judgmental", "Eureka Question"] + CONTROLS

categorical_IVs = [col for col in df_condition_level_data.columns if "categorical" in col] + CONTROLS

# save the lists of IVs
with open('../outputs/iv_lists.pkl', 'wb') as f:
	pickle.dump({
		"basic_IVs": basic_IVs,
		"mcgrath_continuous": mcgrath_continuous,
		"steiner_continuous": steiner_continuous,
		"laughlin_continuous": laughlin_continuous,
		"categorical_IVs": categorical_IVs
	}, f)

Print out the independent variables to verify what they are:

In [15]:
pprint.pprint({
		"basic_IVs": basic_IVs,
		"mcgrath_continuous": mcgrath_continuous,
		"steiner_continuous": steiner_continuous,
		"laughlin_continuous": laughlin_continuous,
		"categorical_IVs": categorical_IVs
	})

{'basic_IVs': ['playerCount',
               'Low',
               'Medium',
               'High',
               'Conceptual-Behavioral',
               'Type 1 (Planning)',
               'Type 2 (Generate)',
               'Type 5 (Cognitive Conflict)',
               'Type 7 (Battle)',
               'Type 8 (Performance)',
               'Divisible-Unitary',
               'Maximizing',
               'Optimizing',
               'Outcome Multiplicity',
               'Solution Scheme Multiplicity',
               'Decision Verifiability',
               'Shared Knowledge',
               'Within-System Solution',
               'Answer Recognizability',
               'Time Solvability',
               'Type 3 and Type 4 (Objective Correctness)',
               'Conflicting Tradeoffs',
               'Solution Scheme Outcome Uncertainty',
               'Eureka Question',
               'Intellectual-Manipulative',
               'Intellective-Judgmental',
               'Creati

Functions to train on one wave and test on another

In [16]:
def get_rmse(y_true, y_pred):
	"""
	Calculate RMSE between true and predicted values.
	"""
	return np.sqrt(np.mean((y_true - y_pred) ** 2))

# function for getting RMSE from a trained model (OLS)
def get_rmse_ols(model_ols, X_test, y_true, ivs):
	X_test_ols = sm.add_constant(X_test[ivs].copy(), has_constant='add')
	y_pred_ols = model_ols.predict(X_test_ols)
	rmse_ols = get_rmse(y_true, y_pred_ols)

	return rmse_ols

In [17]:
def train_wave_a(wave_a_data, dv_type, ivs):
    assert dv_type in ["strong", "weak"]
    X = wave_a_data[ivs]
    X = sm.add_constant(X, has_constant='add')
    y = wave_a_data[dv_type]
    return sm.OLS(y, X).fit()

def custom_r2(y_pred, y_actual, wave_a_data, wave_b_data, dv_type):
    # Compute R^2 on the test set, using the training set as a baseline
    naive_prediction_errs = []
    for i, row in wave_b_data.iterrows():
        
        # get all instances of groups of the same size performing 
        # all the tasks in the training set at the same complexity level.
        playerCount = row["playerCount"]
        wave_a_subset = wave_a_data[(wave_a_data["playerCount"] == playerCount) & 
                                    (wave_a_data["Low"] == row["Low"]) & 
                                    (wave_a_data["Medium"] == row["Medium"]) &
                                    (wave_a_data["High"] == row["High"])]
        y_training = np.mean(wave_a_subset[dv_type])
        # predict the value of the DV (in wave_b) using the mean of the training data (from wave_a)
        y_actual_i = row[dv_type]
        fold_err = (y_actual_i - y_training)**2
        naive_prediction_errs.append(fold_err)

    r2 = 1 - np.sum((y_pred - y_actual)**2) / np.sum(naive_prediction_errs)
    
    return r2

def test_wave_b(wave_a_data, wave_b_data, model, dv_type, ivs):
    assert dv_type in ["strong", "weak"]
    
    X = wave_b_data[ivs]
    X = sm.add_constant(X, has_constant='add')
    y_actual = wave_b_data[dv_type]
    y_pred = model.predict(X)
    r2 = custom_r2(y_pred, y_actual, wave_a_data, wave_b_data, dv_type)
    
    return r2, y_actual, y_pred

def plot_results(y_actual, y_pred, y_train, r2, title="Model Results (Figure 2)"):
    min_val = min(0, 0) # fix at 0,0
    larger_max_val = max(y_actual.max(), y_pred.max())
    round_max_val = round(larger_max_val, 0)
    max_val = max(round_max_val, round_max_val) # the larger max value (between the predicted and the actual)
    
    plt.scatter(y_actual, y_pred)
    plt.axhline(y_train.mean(), color='r', linestyle='-')

    # plot y=x line
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=1)
    plt.text(0.1, 0.9, f"$R^2$ = {r2:.2f}", transform=plt.gca().transAxes, fontsize=20)

    plt.xlim(min_val, max_val) # gotta make sure the lims are the same
    plt.ylim(min_val, max_val)
    plt.xlabel("Observed")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.show()

def train_wave_a_test_wave_b(wave_a_data, wave_b_data, dv_type, ivs, title, plot = False):
    model = train_wave_a(wave_a_data, dv_type, ivs)
    r2, y, y_pred = test_wave_b(wave_a_data, wave_b_data, model, dv_type, ivs)
    if plot:
        # plot the results; defaults to false because it gets a little overwhelming :)
        plot_results(y, y_pred, wave_a_data[dv_type], r2, title)
    return model, r2

### Build our Linear Models

In [18]:
for wave in TRAIN_TEST_DATA.keys():
	for model_type in MODELS.keys():
		for dv_type in ["strong", "weak"]:
			# if the model for this wave and dv_type is not trained yet, train it
			if not MODELS[model_type]["OLS"][wave][dv_type] or not MODEL_R2_RESULTS[model_type]["OLS"][wave][dv_type]:

				# get the training and testing data
				wave_a_data = TRAIN_TEST_DATA[wave]["train"]
				wave_b_data = TRAIN_TEST_DATA[wave]["test"]

				if model_type == "Task Space":
					ivs = basic_IVs
				elif model_type == "McGrath Categorical":
					ivs = categorical_IVs
				elif model_type == "McGrath Subspace":
					ivs = mcgrath_continuous
				elif model_type == "Steiner Subspace":
					ivs = steiner_continuous
				elif model_type == "Laughlin Subspace":
					ivs = laughlin_continuous
				else:
					raise ValueError("Unknown model type: {}".format(model_type))

				# train the model and test it
				model, r2 = train_wave_a_test_wave_b(wave_a_data, wave_b_data, dv_type, ivs, 
													f"{model_type} {dv_type} {wave}", plot=False)
				
				rmse = get_rmse_ols(model, wave_b_data, wave_b_data[dv_type], ivs)
				
				# store the results
				MODELS[model_type]["OLS"][wave][dv_type] = model
				MODEL_R2_RESULTS[model_type]["OLS"][wave][dv_type] = r2
				MODEL_RMSE_RESULTS[model_type]["OLS"][wave][dv_type] = rmse
				save_current_model_and_results_version(MODELS, MODEL_R2_RESULTS, MODEL_RMSE_RESULTS)

# ElasticNet Models

In [19]:
def add_interactions(X):
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    return poly.fit_transform(X)

def get_rmse_for_enet(model_enet, X_test, y_true, ivs):
    X_test_enet = add_interactions(X_test[ivs].copy())
    y_pred_enet = model_enet.predict(X_test_enet)
    rmse_enet = get_rmse(y_true, y_pred_enet)

    return rmse_enet

def train_wave_a_enet(wave_a_data, dv_type, ivs, random_state=19104):
    assert dv_type in ["strong", "weak"]
    X = wave_a_data[ivs]
    X_interactions = add_interactions(X)
    y = wave_a_data[dv_type]
    
    # Define a range of values for alpha and l1_ratio
    alphas = np.logspace(-4, 1, 50)
    l1_ratio = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

    # Initialize and fit ElasticNetCV
    elastic_net_cv = ElasticNetCV(cv=5, alphas=alphas, l1_ratio=l1_ratio, random_state=random_state)
    elastic_net_cv.fit(X_interactions, y)
    
    # Re-fit the model on the full training dataset using the best hyperparameters
    elastic_net = ElasticNet(alpha=elastic_net_cv.alpha_, l1_ratio=elastic_net_cv.l1_ratio_)
    elastic_net.fit(X_interactions, y)
    
    return elastic_net

def test_wave_b_enet(wave_a_data, wave_b_data, model, dv_type, ivs):
    assert dv_type in ["strong", "weak"]
    
    X = wave_b_data[ivs]
    X_interactions = add_interactions(X)
    y_actual = wave_b_data[dv_type]
    y_pred = model.predict(X_interactions)
    
    r2 = custom_r2(y_pred, y_actual, wave_a_data, wave_b_data, dv_type)
    
    return r2, y_actual, y_pred

def train_wave_a_test_wave_b_enet(wave_a_data, wave_b_data, dv_type, ivs, title, plot=False):
    model = train_wave_a_enet(wave_a_data, dv_type, ivs)
    r2, y_actual, y_pred = test_wave_b_enet(wave_a_data, wave_b_data, model, dv_type, ivs)
    if plot:
        plot_results(y_actual, y_pred, wave_a_data[dv_type], r2, title)
    return model, r2

### Build our ENet Models

In [20]:
for wave in tqdm(TRAIN_TEST_DATA.keys(), desc="Waves"):
	for model_type in MODELS.keys():
		for dv_type in ["strong", "weak"]:
			
			if not MODELS[model_type]["E-Net"][wave][dv_type] or not MODEL_R2_RESULTS[model_type]["E-Net"][wave][dv_type]:
				# Get the training and testing data
				wave_a_data = TRAIN_TEST_DATA[wave]["train"]
				wave_b_data = TRAIN_TEST_DATA[wave]["test"]

				if model_type == "Task Space":
					ivs = basic_IVs
				elif model_type == "McGrath Categorical":
					ivs = categorical_IVs
				elif model_type == "McGrath Subspace":
					ivs = mcgrath_continuous
				elif model_type == "Steiner Subspace":
					ivs = steiner_continuous
				elif model_type == "Laughlin Subspace":
					ivs = laughlin_continuous
				else:
					raise ValueError("Unknown model type: {}".format(model_type))

				# Train the model and test it
				model, r2 = train_wave_a_test_wave_b_enet(wave_a_data, wave_b_data, dv_type, ivs, 
														f"{model_type} {dv_type} {wave}", plot=False)
				rmse = get_rmse_for_enet(model, wave_b_data, wave_b_data[dv_type], ivs)
				
				# Store the results
				MODELS[model_type]["E-Net"][wave][dv_type] = model
				MODEL_R2_RESULTS[model_type]["E-Net"][wave][dv_type] = r2
				MODEL_RMSE_RESULTS[model_type]["E-Net"][wave][dv_type] = rmse
				save_current_model_and_results_version(MODELS, MODEL_R2_RESULTS, MODEL_RMSE_RESULTS)

				# Optional: Print progress information
				print(f"Completed: {model_type} - {dv_type} - {wave}")

Waves: 100%|██████████| 2/2 [00:00<00:00, 11765.23it/s]


# Neural Network Models

In [21]:
def create_model(trial: Trial, input_shape):
	# hyperparameters from Mark's comment
	learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-1)
	n_units = trial.suggest_int("n_units", 32, 512)
	n_layers = trial.suggest_categorical("n_layers", [1, 2, 3, 4, 5])
	batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
	dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.3)
	activation = trial.suggest_categorical("activation", ['relu'])

	# Build model
	model = Sequential()
	model.add(Dense(n_units, activation=activation, input_shape=input_shape))
	
	for _ in range(1, n_layers):
		model.add(Dense(n_units, activation=activation))
		model.add(Dropout(dropout_rate))

	model.add(Dense(1))
	
	model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
				   loss='mean_squared_error')
	
	return model, batch_size

def leave_one_task_out(df, model_func, iv, dv):
	# assert that dv is one of 'strong' or 'weak'
	assert dv in ["strong", "weak"]

	tasks = df["task"].unique()
	total_left_out_loss = 0

	for task in tasks:
		train_data = df[df["task"] != task]
		test_data = df[df["task"] == task]
		
		X_train, y_train = train_data[iv], train_data[dv]
		X_test, y_test = test_data[iv], test_data[dv]

		input_shape = (X_train.shape[1],)
		model, batch_size = model_func(input_shape)
		
		model.fit(X_train, y_train, batch_size=batch_size, epochs=100, verbose=0)
		loss = model.evaluate(X_test, y_test, verbose=0)
		
		total_left_out_loss += loss

	return total_left_out_loss

"""Define objectives"""
def objective(trial, wave_data, iv, dv):
	def model_func(input_shape):
		return create_model(trial, input_shape)
	total_loss = leave_one_task_out(wave_data, model_func, iv=iv, dv=dv)
	return total_loss

def optimize_and_save(study_name, wave_data, iv, dv):
	try:
		# Load the hyperparameters from the .pkl file
		with open(f"../outputs/best_param_pkls/best_params_{study_name}.pkl", "rb") as f:
			best_params = pickle.load(f)
	except FileNotFoundError:
		study = create_study(direction="minimize")
		study.optimize(lambda trial: objective(trial, wave_data, iv, dv), n_trials=100)

		best_trial = study.best_trial
		best_params = best_trial.params
		# Save the hyperparameters to a .pkl file
		with open(f"../outputs/best_param_pkls/best_params_{study_name}.pkl", "wb") as f:
			pickle.dump(best_params, f)

	return best_params

def set_random_seeds(seed_value=19104):
	random.seed(seed_value)
	np.random.seed(seed_value)
	tf.random.set_seed(seed_value)

def create_best_model(input_shape, best_params):    
	learning_rate = best_params['learning_rate']
	n_units = best_params['n_units']
	n_layers = best_params['n_layers']
	batch_size = best_params['batch_size']
	dropout_rate = best_params['dropout_rate']
	activation = best_params['activation']
	
	# Build model
	model = Sequential()
	model.add(Dense(n_units, activation=activation, input_shape=input_shape))
	
	for _ in range(1, n_layers):
		model.add(Dense(n_units, activation=activation))
		model.add(Dropout(dropout_rate))
	
	model.add(Dense(1))
	model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
				  loss='mean_squared_error')
	
	return model, batch_size

In [22]:
def train_wave_a_nn(wave_a_data, ivs, dv_type):
	assert dv_type in ["strong", "weak"]

	# Random seed for reproducibility
	set_random_seeds()

	# Objective function for Optuna
	def objective(trial):
		return leave_one_task_out(wave_a_data, lambda shape: create_model(trial, shape), ivs, dv_type)

	# Hyperparameter optimization
	study = create_study(direction="minimize")
	study.optimize(objective, n_trials=100)

	best_params = study.best_trial.params

	input_shape = (wave_a_data[ivs].shape[1],)
	model, batch_size = create_best_model(input_shape, best_params)
	
	X_train = wave_a_data[ivs]
	y_train = wave_a_data[dv_type]

	model.fit(X_train, y_train, batch_size=batch_size, epochs=100, verbose=0)

	print(f"Best hyperparameters: {best_params}")

	return model

def test_wave_b_nn(wave_a_data, wave_b_data, model, ivs, dv_type):
	X_new = wave_b_data[ivs]
	y_actual = wave_b_data[dv_type]

	# Ensure consistent shape
	if len(X_new.shape) == 1:
		X_new = X_new.reshape(-1, 1)

	y_pred = model.predict(X_new).flatten()

	r2 = custom_r2(y_pred, y_actual, wave_a_data, wave_b_data, dv_type)

	return r2, y_actual, y_pred

### Build our NN models

In [23]:
for wave in tqdm(TRAIN_TEST_DATA.keys(), desc="Waves"):
	# Note: only do this for Task Space + McGrath baselines
	for model_type in tqdm(['Task Space', 'McGrath Categorical', 'McGrath Subspace'], desc="Model Types", leave=False):
		for dv_type in tqdm(["strong", "weak"], desc="DV Types", leave=False):

			if not MODELS[model_type]["NN"][wave][dv_type] or not MODEL_R2_RESULTS[model_type]["NN"][wave][dv_type]:
				wave_a_data = TRAIN_TEST_DATA[wave]["train"]
				wave_b_data = TRAIN_TEST_DATA[wave]["test"]

				if model_type == "Task Space":
					ivs = basic_IVs
				elif model_type == "McGrath Categorical":
					ivs = categorical_IVs
				elif model_type == "McGrath Subspace":
					ivs = mcgrath_continuous
				elif model_type == "Steiner Subspace":
					ivs = steiner_continuous
				elif model_type == "Laughlin Subspace":
					ivs = laughlin_continuous
				else:
					raise ValueError("Unknown model type: {}".format(model_type))

				model = train_wave_a_nn(wave_a_data, ivs, dv_type)
				r2, y_actual, y_pred = test_wave_b_nn(wave_a_data, wave_b_data, model, ivs, dv_type)
				rmse = get_rmse(y_actual, y_pred)       

				MODELS[model_type]["NN"][wave][dv_type] = model
				MODEL_R2_RESULTS[model_type]["NN"][wave][dv_type] = r2
				MODEL_RMSE_RESULTS[model_type]["NN"][wave][dv_type] = rmse
				save_current_model_and_results_version(MODELS, MODEL_R2_RESULTS, MODEL_RMSE_RESULTS)

				print(f"Completed: {model_type} - {dv_type} - {wave}")

Waves:   0%|          | 0/2 [00:00<?, ?it/s]











Waves: 100%|██████████| 2/2 [00:00<00:00, 202.39it/s]


### All Model Results

In [24]:
def create_r2_dataframe(results_dict, dv_type):
	data = []

	for iv_type, models in results_dict.items():
		for model_type, waves in models.items():
			for wave, dv_values in waves.items():
				if dv_values[dv_type] is not None:
					data.append({
						'IV Type': iv_type,
						'Training Wave': wave,
						'Model': model_type,
						'R^2': dv_values[dv_type]
					})
	
	df = pd.DataFrame(data)
	df['R^2'] = df['R^2'].round(2)

	# Order the 'Training Wave' column
	wave_order = ['Wave 1', 'Wave 2']
	df['Training Wave'] = pd.Categorical(df['Training Wave'], categories=wave_order, ordered=True)

	df_pivot = df.pivot_table(values='R^2', index=['IV Type', 'Training Wave'], columns='Model')
	
	return df_pivot

print("R^2 Results for Strong Group Advantage:")
display(create_r2_dataframe(MODEL_R2_RESULTS, "strong"))
print("R^2 Results for Weak Group Advantage:")
display(create_r2_dataframe(MODEL_R2_RESULTS, "weak"))

print("RMSE Results for Strong Group Advantage:")
display(create_r2_dataframe(MODEL_RMSE_RESULTS, "strong"))
print("RMSE Results for Weak Group Advantage:")
display(create_r2_dataframe(MODEL_RMSE_RESULTS, "weak"))

R^2 Results for Strong Group Advantage:


Model                              E-Net    NN   OLS
IV Type             Training Wave                   
Laughlin Subspace   Wave 1          0.22   NaN -0.39
                    Wave 2          0.28   NaN  0.12
McGrath Categorical Wave 1         -0.31 -0.29 -0.28
                    Wave 2         -0.02 -0.05 -0.03
McGrath Subspace    Wave 1          0.21  0.20  0.23
                    Wave 2          0.31  0.33  0.21
Steiner Subspace    Wave 1          0.01   NaN -0.10
                    Wave 2          0.15   NaN  0.10
Task Space          Wave 1          0.40  0.41  0.43
                    Wave 2          0.43  0.37  0.33

R^2 Results for Weak Group Advantage:


Model                              E-Net    NN   OLS
IV Type             Training Wave                   
Laughlin Subspace   Wave 1          0.02   NaN -1.13
                    Wave 2          0.03   NaN -0.20
McGrath Categorical Wave 1          0.02 -0.28 -0.05
                    Wave 2         -0.03 -0.15  0.06
McGrath Subspace    Wave 1          0.10  0.09 -0.62
                    Wave 2         -0.02 -0.16  0.06
Steiner Subspace    Wave 1          0.02   NaN  0.05
                    Wave 2          0.22   NaN  0.16
Task Space          Wave 1          0.02  0.20  0.48
                    Wave 2          0.40  0.32  0.35

RMSE Results for Strong Group Advantage:


Model                              E-Net    NN   OLS
IV Type             Training Wave                   
Laughlin Subspace   Wave 1          0.29   NaN  0.39
                    Wave 2          0.31   NaN  0.35
McGrath Categorical Wave 1          0.38  0.37  0.37
                    Wave 2          0.37  0.38  0.38
McGrath Subspace    Wave 1          0.29  0.30  0.29
                    Wave 2          0.31  0.30  0.33
Steiner Subspace    Wave 1          0.33   NaN  0.35
                    Wave 2          0.34   NaN  0.35
Task Space          Wave 1          0.26  0.25  0.25
                    Wave 2          0.28  0.29  0.30

RMSE Results for Weak Group Advantage:


Model                              E-Net    NN   OLS
IV Type             Training Wave                   
Laughlin Subspace   Wave 1          0.74   NaN  1.09
                    Wave 2          0.74   NaN  0.82
McGrath Categorical Wave 1          0.74  0.85  0.77
                    Wave 2          0.77  0.81  0.73
McGrath Subspace    Wave 1          0.71  0.71  0.95
                    Wave 2          0.76  0.81  0.73
Steiner Subspace    Wave 1          0.74   NaN  0.73
                    Wave 2          0.67   NaN  0.69
Task Space          Wave 1          0.74  0.67  0.54
                    Wave 2          0.58  0.62  0.61

# Boostrapped Robustness Check

In [25]:
# try to read in BOOTSTRAP_R2_RESULTS if it exists, otherwise create it
bootstrap_r2_pkl_filename = '../outputs/bootstrap_r2_results.pkl'
bootstrap_model_pkl_filename = '../outputs/bootstrap_models.pkl'
bootstrap_rmse_pkl_filename = '../outputs/bootstrap_rmse_results.pkl'

try:
	with open(bootstrap_r2_pkl_filename, 'rb') as f:
		BOOTSTRAP_R2_RESULTS = pickle.load(f)
	with open(bootstrap_model_pkl_filename, 'rb') as f:
		BOOTSTRAP_MODELS = pickle.load(f)
	with open(bootstrap_rmse_pkl_filename, 'rb') as f:
		BOOTSTRAP_RMSE_RESULTS = pickle.load(f)
except FileNotFoundError:
	base_results_by_dv = {
			"OLS": {"strong": [], "weak": []},
			"E-Net": {"strong": [], "weak": []},
			"NN": {"strong": [], "weak": []}
		}

	BOOTSTRAP_R2_RESULTS = {
			"Task Space": copy.deepcopy(base_results_by_dv),
			"McGrath Categorical": copy.deepcopy(base_results_by_dv),
			"McGrath Subspace": copy.deepcopy(base_results_by_dv),
			"Steiner Subspace": copy.deepcopy(base_results_by_dv),
			"Laughlin Subspace": copy.deepcopy(base_results_by_dv)
		}
	BOOTSTRAP_MODELS = copy.deepcopy(BOOTSTRAP_R2_RESULTS)
	BOOTSTRAP_RMSE_RESULTS = copy.deepcopy(BOOTSTRAP_R2_RESULTS)

# all ways of choosing 5 tasks from df_condition_level_data
def get_all_combinations_of_tasks(df, num_tasks=5):
	"""
	Get all combinations of tasks from the DataFrame.
	"""
	tasks = df['task'].unique()
	return list(itertools.combinations(tasks, num_tasks))

# Sample a limited number of task combinations
all_held_out_options = get_all_combinations_of_tasks(df_condition_level_data, num_tasks=5)
random.seed(19104)
num_to_sample = 250
held_out_sample = random.sample(all_held_out_options, num_to_sample)

In [26]:
# function for getting RMSE from a trained model
def get_rmse_for_ols_enet(model_ols, model_enet, X_test, y_true, ivs):
	"""
	Calculate RMSE for the OLS and E-Net predictions.
	"""
	rmse_ols = get_rmse_ols(model_ols, X_test, y_true, ivs)
	rmse_enet = get_rmse_for_enet(model_enet, X_test, y_true, ivs)
	
	return rmse_ols, rmse_enet

In [27]:
for i, held_out_set in enumerate(tqdm(held_out_sample, desc="Bootstrapping R2 Results", total=len(held_out_sample))):
	
	wave_a_data = df_condition_level_data[~df_condition_level_data["task"].isin(held_out_set)]
	wave_b_data = df_condition_level_data[df_condition_level_data["task"].isin(held_out_set)]

	for model_type in MODELS.keys():

		for dv_type in ["strong", "weak"]:
			if model_type == "Task Space":
				ivs = basic_IVs
			elif model_type == "McGrath Categorical":
				ivs = categorical_IVs
			elif model_type == "McGrath Subspace":
				ivs = mcgrath_continuous
			elif model_type == "Steiner Subspace":
				ivs = steiner_continuous
			elif model_type == "Laughlin Subspace":
				ivs = laughlin_continuous
			else:
				raise ValueError("Unknown model type: {}".format(model_type))
			

			# if the BOOTSTRAP_R2_RESULTS already has a model at this index, skip it
			try:
				model_ols = BOOTSTRAP_MODELS[model_type]["OLS"][dv_type][i]
				r2_ols = BOOTSTRAP_MODELS[model_type]["E-Net"][dv_type][i]
			except IndexError:
				
				# Train and test OLS model
				model_ols, r2_ols = train_wave_a_test_wave_b(
					wave_a_data.copy(), wave_b_data.copy(), dv_type, ivs, f"{model_type} {dv_type} {held_out_set}", plot=False
				)
				# Train and test E-Net model
				model_enet, r2_enet = train_wave_a_test_wave_b_enet(
					wave_a_data.copy(), wave_b_data.copy(), dv_type, ivs, f"{model_type} {dv_type} {held_out_set}", plot=False
				)

				# get model rmse on test set
				rmse_ols, rmse_enet = get_rmse_for_ols_enet(
					model_ols, model_enet, wave_b_data, wave_b_data[dv_type], ivs
				)
				
				# store the models
				BOOTSTRAP_MODELS[model_type]["OLS"][dv_type].append(model_ols)
				BOOTSTRAP_MODELS[model_type]["E-Net"][dv_type].append(model_enet)

				# Store the R^2
				BOOTSTRAP_R2_RESULTS[model_type]["OLS"][dv_type].append(r2_ols)
				BOOTSTRAP_R2_RESULTS[model_type]["E-Net"][dv_type].append(r2_enet)

				# Store the RMSE
				BOOTSTRAP_RMSE_RESULTS[model_type]["OLS"][dv_type].append(rmse_ols)
				BOOTSTRAP_RMSE_RESULTS[model_type]["E-Net"][dv_type].append(rmse_enet)

				# write to pkl
				with open(bootstrap_model_pkl_filename, 'wb') as f:
					pickle.dump(BOOTSTRAP_MODELS, f)
				with open(bootstrap_r2_pkl_filename, 'wb') as f:
					pickle.dump(BOOTSTRAP_R2_RESULTS, f)
				with open(bootstrap_rmse_pkl_filename, 'wb') as f:
					pickle.dump(BOOTSTRAP_RMSE_RESULTS, f)

Bootstrapping R2 Results: 100%|██████████| 250/250 [00:00<00:00, 4741.77it/s]


## Compare Bootstrap Results

In [28]:
def calculate_pairwise_differences(data1, data2):
	differences = np.array(data1) - np.array(data2)
	mean_diff = np.mean(differences)
	
	# 1.96 method for confidence intervals
	sem_diff = np.std(differences, ddof=1) / np.sqrt(len(differences))
	ci_1_96_lower_diff = mean_diff - 1.96 * sem_diff
	ci_1_96_upper_diff = mean_diff + 1.96 * sem_diff
	
	# Percentile method
	ci_percentile_lower_diff = np.percentile(differences, 2.5)
	ci_percentile_upper_diff = np.percentile(differences, 97.5)
	
	return mean_diff, ci_1_96_lower_diff, ci_1_96_upper_diff, ci_percentile_lower_diff, ci_percentile_upper_diff

def generate_pairwise_summary_table(bootstrapped_r2_results, dv_choice):
	summary_data = []

	for model_type in ["OLS", "E-Net"]:
		iv_types = list(bootstrapped_r2_results.keys())
		for i, iv_1 in enumerate(iv_types):
			if not bootstrapped_r2_results[iv_1][model_type].get(dv_choice, []):
				continue
			for iv_2 in iv_types[i+1:]:
				if not bootstrapped_r2_results[iv_2][model_type].get(dv_choice, []):
					continue
				data1 = bootstrapped_r2_results[iv_1][model_type][dv_choice]
				data2 = bootstrapped_r2_results[iv_2][model_type][dv_choice]
				if data1 and data2:
					(mean_diff, ci_1_96_lower_diff, ci_1_96_upper_diff,
					 ci_percentile_lower_diff, ci_percentile_upper_diff) = calculate_pairwise_differences(data1, data2)

					comparison_name = f"{iv_1} - {iv_2}"
					summary_data.append({
						'Model Type': model_type,
						'Comparison': comparison_name,
						'Mean Diff': mean_diff,
						'CI 1.96 Lower': ci_1_96_lower_diff,
						'CI 1.96 Upper': ci_1_96_upper_diff,
						'CI Percentile Lower': ci_percentile_lower_diff,
						'CI Percentile Upper': ci_percentile_upper_diff
					})

	df_summary = pd.DataFrame(summary_data)
	df_summary.sort_values(by='Mean Diff', ascending=False, inplace=True)
	return df_summary

# Generate and print tables for both 'strong' and 'weak' DV types
pairwise_table_strong = generate_pairwise_summary_table(BOOTSTRAP_R2_RESULTS, 'strong')
pairwise_table_weak = generate_pairwise_summary_table(BOOTSTRAP_R2_RESULTS, 'weak')

print("Pairwise R^2 Summary Table - Strong Group Advantage")
print(pairwise_table_strong.to_string(index=False))
print("\nPairwise R^2 Summary Table - Weak Group Advantage")
print(pairwise_table_weak.to_string(index=False))

Pairwise R^2 Summary Table - Strong Group Advantage
Model Type                              Comparison  Mean Diff  CI 1.96 Lower  CI 1.96 Upper  CI Percentile Lower  CI Percentile Upper
       OLS    Steiner Subspace - Laughlin Subspace   0.880025       0.610838       1.149213            -1.040674             6.424184
       OLS McGrath Categorical - Laughlin Subspace   0.615070       0.307635       0.922506            -1.624165             6.700112
     E-Net          Task Space - Laughlin Subspace   0.336575       0.225957       0.447194            -1.016065             2.333436
       OLS  McGrath Categorical - McGrath Subspace   0.322073       0.067844       0.576302            -1.905212             7.216360
       OLS    McGrath Subspace - Laughlin Subspace   0.292997      -0.052211       0.638205            -4.677030             6.517318
     E-Net        Task Space - McGrath Categorical   0.273171       0.159118       0.387225            -2.195947             1.635501
     E-Net

In [29]:
# Generate and print tables for both 'strong' and 'weak' DV types
pairwise_table_strong_rmse = generate_pairwise_summary_table(BOOTSTRAP_RMSE_RESULTS, 'strong')
pairwise_table_weak_rmse = generate_pairwise_summary_table(BOOTSTRAP_RMSE_RESULTS, 'weak')

print("Pairwise RMSE Summary Table - Strong Group Advantage")
print(pairwise_table_strong_rmse.to_string(index=False))
print("\nPairwise RMSE Summary Table - Weak Group Advantage")
print(pairwise_table_weak_rmse.to_string(index=False))

Pairwise RMSE Summary Table - Strong Group Advantage
Model Type                              Comparison  Mean Diff  CI 1.96 Lower  CI 1.96 Upper  CI Percentile Lower  CI Percentile Upper
       OLS           Task Space - Steiner Subspace   0.105917       0.089952       0.121882            -0.076179             0.410932
       OLS        Task Space - McGrath Categorical   0.063681       0.047049       0.080312            -0.146280             0.393367
       OLS           Task Space - McGrath Subspace   0.060425       0.042637       0.078213            -0.232999             0.383301
       OLS     McGrath Subspace - Steiner Subspace   0.045492       0.029677       0.061307            -0.141888             0.409262
       OLS  McGrath Categorical - Steiner Subspace   0.042236       0.032375       0.052098            -0.105855             0.195264
       OLS          Task Space - Laughlin Subspace   0.040732       0.021136       0.060328            -0.281593             0.352871
     E-Ne

# Check Model Robustness

Are the OLS/E-Net Model Results Robust to Noise?

In [30]:
task_vars = sorted(list(set(basic_IVs).difference(set(CONTROLS)))) # sorted creates a consistent ordering

In [ ]:
# create a data structure to store the noise check
try:
	# read the models and results from the previous save
	with open('../outputs/enet_noise_robustness.pkl', 'rb') as f:
		ENETS_NOISE_ROBUSTNESS = pickle.load(f)
	with open('../outputs/enet_noise_robustness_r2_results.pkl', 'rb') as f:
		ENETS_NOISE_ROBUSTNESS_R2_RESULTS = pickle.load(f)
	with open('../outputs/enet_noise_robustness_rmse_results.pkl', 'rb') as f:
		ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS = pickle.load(f)

except FileNotFoundError:
	# if the file is not found, we will create the MODELS and MODEL_R2_RESULTS dictionaries
	ENETS_NOISE_ROBUSTNESS = {}
	for i in range(len(task_vars)+1):
		ENETS_NOISE_ROBUSTNESS[i] = { # 1 entry per num tasks cols affected x noise level x dv (strong v. weak)
			0.25: {"strong": None, "weak": None},
			0.5: {"strong": None, "weak": None},
			0.75: {"strong": None, "weak": None},
			1: {"strong": None, "weak": None}
		}
	ENETS_NOISE_ROBUSTNESS_R2_RESULTS = copy.deepcopy(ENETS_NOISE_ROBUSTNESS)
	ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS = copy.deepcopy(ENETS_NOISE_ROBUSTNESS)

# save the dictionaries
def save_noise_dicts(ENETS_NOISE_ROBUSTNESS, ENETS_NOISE_ROBUSTNESS_R2_RESULTS, ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS):
	with open('../outputs/enet_noise_robustness.pkl', 'wb') as f:
		pickle.dump(ENETS_NOISE_ROBUSTNESS, f)
	with open('../outputs/enet_noise_robustness_r2_results.pkl', 'wb') as f:
		pickle.dump(ENETS_NOISE_ROBUSTNESS_R2_RESULTS, f)
	with open('../outputs/enet_noise_robustness_rmse_results.pkl', 'wb') as f:
		pickle.dump(ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS, f)

save_noise_dicts(ENETS_NOISE_ROBUSTNESS, ENETS_NOISE_ROBUSTNESS_R2_RESULTS, ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS)

In [46]:
random.seed(19104)

noise_levels_sd = [0.25, 0.5, 0.75, 1]

for num_to_perturb in tqdm(range(len(task_vars)+1), desc="Perturbing dimensions..."):
    for noise_level in noise_levels_sd:

        # train: Waves 1-2; test: Wave 3
        train_data = TRAIN_TEST_DATA["Wave 2"]["train"].copy() # copy so we can modify
        test_data = TRAIN_TEST_DATA["Wave 2"]["test"]

        # randomly add noise to the Task Space dimensions (just on the training data)
        random.shuffle(task_vars)
        cols_to_preturb = random.choices(task_vars, k=num_to_perturb)

        for col in cols_to_preturb:
            train_data[col] += np.random.normal(0, noise_level, train_data.shape[0])

        for dv_type in ["strong", "weak"]:   

            if ENETS_NOISE_ROBUSTNESS[num_to_perturb][noise_level][dv_type] is None:
                # Then train on the preturbed data
                model, r2 = train_wave_a_test_wave_b_enet(train_data, test_data, dv_type, basic_IVs, 
                                                                        f"{model_type} {dv_type} {wave} with {num_to_perturb} features perturbed with noise {noise_level}", plot=False)
                rmse = get_rmse_for_enet(model, test_data, test_data[dv_type], basic_IVs)

                ENETS_NOISE_ROBUSTNESS[num_to_perturb][noise_level][dv_type] = model
                ENETS_NOISE_ROBUSTNESS_R2_RESULTS[num_to_perturb][noise_level][dv_type] = r2
                ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS[num_to_perturb][noise_level][dv_type] = rmse

                save_noise_dicts(ENETS_NOISE_ROBUSTNESS, ENETS_NOISE_ROBUSTNESS_R2_RESULTS, ENETS_NOISE_ROBUSTNESS_RMSE_RESULTS)
                print(f"Completed: perturbing {num_to_perturb} features - noise level {noise_level} - dv: {dv_type}")

Perturbing dimensions...:   0%|          | 0/25 [00:00<?, ?it/s]

Completed: perturbing 24 features - noise level 0.25 - dv: strong
Completed: perturbing 24 features - noise level 0.25 - dv: weak
Completed: perturbing 24 features - noise level 0.5 - dv: strong
Completed: perturbing 24 features - noise level 0.5 - dv: weak
Completed: perturbing 24 features - noise level 0.75 - dv: strong
Completed: perturbing 24 features - noise level 0.75 - dv: weak
Completed: perturbing 24 features - noise level 1 - dv: strong


Perturbing dimensions...: 100%|██████████| 25/25 [03:22<00:00,  8.10s/it]

Completed: perturbing 24 features - noise level 1 - dv: weak
